## Pyspark Assignment
### <i> Group 13 </i>

### Food

##### i.Extract:  Load the data
   - Read data as csv via spark dataframe

##### ii.Transform: Exploratory data analysis using spark df
    - Unique records count
    - select columns GPA	Gender	breakfast	calories_chicken	calories_day	calories_scone	coffee type_sports
    - handling nan -> replace with 0 for numerical columns and Others in categorical columns
    - Replace - Gender -> 1 to M and 2 to F
    - Update : Normalize GPA column - normalized  v = v - min(GPA)/max(GPA)-min(GPA)
    - show schema
    - show all df
    - GroupBy type_sport and sum of cofee

##### iii.Load: Save analysis report
    - show all df, save as files

In [23]:
# Importing necessary libraries
import os
import spark
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession, types
from pyspark.sql.types import StringType
from pyspark.sql.functions import regexp_replace, split, explode, struct, col, concat, lit, when
import pandas as pd
from pyspark.sql.types import DoubleType

In [2]:
os.environ['PYSPARK SUBMIT ARGS'] = " --master local[2] pyspark-shell"
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_231'

In [3]:
conf = SparkConf().setAppName("test").setMaster("local")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
print(sc)
print(spark)

<SparkContext master=local appName=test>


#### 1. Load Data

In [4]:
df = pd.read_csv("C:/Users/Abhijith P K/OneDrive/Desktop/Food/food_coded.csv")

In [5]:
df.head(7)

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,...,soup,sports,thai_food,tortilla_calories,turkey_calories,type_sports,veggies_day,vitamins,waffle_calories,weight
0,2.400,2,1,430,NaN,315.0,1,none,we dont have comfort,9.0,...,1.0,1.0,1,1165.0,345,car racing,5,1,1315,187
1,3.654,1,1,610,3.0,420.0,2,"chocolate, chips, ice cream","Stress, bored, anger",1.0,...,1.0,1.0,2,725.0,690,Basketball,4,2,900,155
2,3.200,1,1,430,3.0,420.0,2,"Pizza, Mac and cheese, ice cream",Boredom,2.0,...,1.0,2.0,5,725.0,690,NaN,3,1,1315,"Not sure, 240"
3,3.500,1,1,720,2.0,420.0,2,"Ice cream, chocolate, chips","Stress, boredom, cravings",1.0,...,1.0,1.0,4,940.0,500,Softball,4,2,760,190
4,2.250,1,1,610,3.0,980.0,2,"Candy, brownies and soda.","None, i don't eat comfort food. I just eat whe...",4.0,...,1.0,2.0,4,940.0,345,None.,1,2,1315,190
5,3.800,2,1,610,3.0,420.0,2,"Chocolate, ice cream, french fries, pretzels","stress, boredom",1.0,...,1.0,1.0,5,940.0,690,soccer,4,1,1315,180
6,3.300,1,1,720,3.0,420.0,1,"Ice cream, cheeseburgers, chips.",I eat comfort food when im stressed out from s...,1.0,...,1.0,2.0,1,725.0,500,none,4,2,1315,137


In [6]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

In [7]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")
food=spark.createDataFrame(df) 

In [9]:
type(food)

pyspark.sql.dataframe.DataFrame

In [10]:
header=food.first()

In [11]:
print(header)

Row(GPA=2.4, Gender=2, breakfast=1, calories_chicken=430, calories_day=None, calories_scone=315.0, coffee=1, comfort_food='none', comfort_food_reasons='we dont have comfort ', comfort_food_reasons_coded=9.0, cook=2.0, comfort_food_reasons_coded.1=9, cuisine=None, diet_current='eat good and exercise', diet_current_coded=1, drink=1.0, eating_changes='eat faster ', eating_changes_coded=1, eating_changes_coded1=1, eating_out=3, employment=3.0, ethnic_food=1, exercise=1.0, father_education=5.0, father_profession='profesor ', fav_cuisine='Arabic cuisine', fav_cuisine_coded=3, fav_food=1.0, food_childhood='rice  and chicken ', fries=2, fruit_day=5, grade_level=2, greek_food=5, healthy_feeling=2, healthy_meal='looks not oily ', ideal_diet='being healthy ', ideal_diet_coded=8, income=5.0, indian_food=5, italian_food=5, life_rewarding=1.0, marital_status=1.0, meals_dinner_friend='rice, chicken,  soup', mother_education=1.0, mother_profession='unemployed', nutritional_check=5, on_off_campus=1.0, 

In [12]:
# To see the dataframe
type(food)

pyspark.sql.dataframe.DataFrame

####  ii. Transform: Exploratory data analysis using spark df

#### Unique RECORD count

In [13]:
food.distinct().count()

123

#### Select columns GPA,Gender,	breakfast,Calories_chicken,	calories_day,	calories_scone,	coffee type_sports

In [14]:
from pyspark.sql.functions import col
food.select(col("GPA"),col("Gender"),col("breakfast"),col("calories_chicken"),col("calories_day"),col("calories_scone"),col("coffee"),col("type_sports")).show()

+-----+------+---------+----------------+------------+--------------+------+--------------------+
|  GPA|Gender|breakfast|calories_chicken|calories_day|calories_scone|coffee|         type_sports|
+-----+------+---------+----------------+------------+--------------+------+--------------------+
|  2.4|     2|        1|             430|        null|         315.0|     1|          car racing|
|3.654|     1|        1|             610|         3.0|         420.0|     2|         Basketball |
|  3.2|     1|        1|             430|         3.0|         420.0|     2|                null|
|  3.5|     1|        1|             720|         2.0|         420.0|     2|            Softball|
| 2.25|     1|        1|             610|         3.0|         980.0|     2|               None.|
|  3.8|     2|        1|             610|         3.0|         420.0|     2|              soccer|
|  3.3|     1|        1|             720|         3.0|         420.0|     1|                none|
|  3.3|     1|      

#### Handling nan -> replace with 0 for numerical columns and others in categorical columns

In [24]:
column_int = [field.name for field in food.schema.fields if isinstance(
    field.dataType, DoubleType)]

In [25]:
#  replace with 0 for numerical columns 
food = food.fillna(0,subset=column_int)

In [26]:
column_str = [field.name for field in food.schema.fields if isinstance(
    field.dataType, StringType)]

In [27]:
# nan -> replace with 'others' in categorical columns
food = food.fillna('Others',subset=column_str)

#### Replace - Gender -> 1 to M and 2 to F

In [28]:
from pyspark.sql.functions import when
food.withColumn('Gender', 
    when(food.Gender.endswith('1'),regexp_replace(food.Gender,'1','M')) \
   .when(food.Gender.endswith('2'),regexp_replace(food.Gender,'2','F')) \
   .otherwise(food.Gender)) \
   .show(truncate=False)

+-----+------+---------+----------------+------------+--------------+------+---------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------+--------------------------+----+----------------------------+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+---------------------+----------+----------+-----------+--------+----------------+----------------------------+-------------------------------------+-----------------+--------+------------------------------------

#### Normalize GPA column - normalized  v = v - min(GPA)/max(GPA)-min(GPA)

In [29]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

# Iterating over columns to be scaled
for i in ["GPA"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")
    pipeline = Pipeline(stages=[assembler, scaler])

    # Fitting pipeline on dataframe
    food = pipeline.fit(food).transform(food).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")

print("After Scaling :")
food

After Scaling :


DataFrame[GPA: double, Gender: bigint, breakfast: bigint, calories_chicken: bigint, calories_day: double, calories_scone: double, coffee: bigint, comfort_food: string, comfort_food_reasons: string, comfort_food_reasons_coded: double, cook: double, comfort_food_reasons_coded.1: bigint, cuisine: double, diet_current: string, diet_current_coded: bigint, drink: double, eating_changes: string, eating_changes_coded: bigint, eating_changes_coded1: bigint, eating_out: bigint, employment: double, ethnic_food: bigint, exercise: double, father_education: double, father_profession: string, fav_cuisine: string, fav_cuisine_coded: bigint, fav_food: double, food_childhood: string, fries: bigint, fruit_day: bigint, grade_level: bigint, greek_food: bigint, healthy_feeling: bigint, healthy_meal: string, ideal_diet: string, ideal_diet_coded: bigint, income: double, indian_food: bigint, italian_food: bigint, life_rewarding: double, marital_status: double, meals_dinner_friend: string, mother_education: dou

In [30]:
food.show(5)

+-----+------+---------+----------------+------------+--------------+------+--------------------+--------------------+--------------------------+----+----------------------------+-------+--------------------+------------------+-----+--------------------+--------------------+---------------------+----------+----------+-----------+--------+----------------+-----------------+--------------+-----------------+--------+--------------------+-----+---------+-----------+----------+---------------+--------------------+--------------------+----------------+------+-----------+------------+--------------+--------------+--------------------+----------------+--------------------+-----------------+-------------+------------+------------+------------+----------------------+----+------+---------+-----------------+---------------+-----------+-----------+--------+---------------+-------------+----------+
|  GPA|Gender|breakfast|calories_chicken|calories_day|calories_scone|coffee|        comfort_food|comfo

#### GroupBy type_sport and  cofee

In [31]:
#group by
file = food.groupBy('type_sports', 'coffee').count()
file.show()

+--------------------+------+-----+
|         type_sports|coffee|count|
+--------------------+------+-----+
|          car racing|     1|    1|
|             rowing |     2|    1|
|  Tennis, Basketball|     2|    1|
|             Dancing|     2|    1|
|                crew|     2|    1|
|     Gaelic Football|     2|    1|
|      none organized|     2|    1|
|         Basketball |     2|    1|
|   Volleyball, Track|     2|    1|
|             Fotball|     2|    1|
|       Marching Band|     2|    1|
|              Hockey|     1|    2|
|no particular eng...|     1|    1|
|              Tennis|     1|    2|
|No, I don't play ...|     2|    1|
|            Lacrosse|     2|    2|
|          volleyball|     2|    1|
|                none|     1|    3|
|   horse back riding|     1|    1|
|              Hockey|     2|    5|
+--------------------+------+-----+
only showing top 20 rows



#### Show schema

In [32]:
food.printSchema()

root
 |-- GPA: double (nullable = false)
 |-- Gender: long (nullable = true)
 |-- breakfast: long (nullable = true)
 |-- calories_chicken: long (nullable = true)
 |-- calories_day: double (nullable = false)
 |-- calories_scone: double (nullable = false)
 |-- coffee: long (nullable = true)
 |-- comfort_food: string (nullable = false)
 |-- comfort_food_reasons: string (nullable = false)
 |-- comfort_food_reasons_coded: double (nullable = false)
 |-- cook: double (nullable = false)
 |-- comfort_food_reasons_coded.1: long (nullable = true)
 |-- cuisine: double (nullable = false)
 |-- diet_current: string (nullable = false)
 |-- diet_current_coded: long (nullable = true)
 |-- drink: double (nullable = false)
 |-- eating_changes: string (nullable = false)
 |-- eating_changes_coded: long (nullable = true)
 |-- eating_changes_coded1: long (nullable = true)
 |-- eating_out: long (nullable = true)
 |-- employment: double (nullable = false)
 |-- ethnic_food: long (nullable = true)
 |-- exercise: 

#### iii. Load: Save analysis report

In [33]:
file = file.toPandas()
file.to_csv('Output.txt', index=False)
print('Successfully saved at ',os.getcwd())

Successfully saved at  C:\Users\Abhijith P K\Downloads
